In [18]:
import sys
import os
import pandas as pd
sys.path.append('/Users/rishabhbhardwaj/Documents/personal/courses/cse6250/project/Chest-XRay-Classification/')

In [34]:
from keras.models import load_model
from keras.preprocessing import image
from src.models import DenseNet, ModelFactory
import numpy as np
import keras
from sklearn.metrics.ranking import roc_auc_score
import sklearn.metrics as metrics

In [4]:
img_width, img_height = 224, 224
model_file_path = '../weights/best_weights_1555982768.7076797.h5'
data_dir = '/Users/rishabhbhardwaj/Documents/personal/courses/cse6250/project/'
valid_file = os.path.join(data_dir, 'CheXpert-v1.0-small/valid.csv')
# model_file_path = '../weights/best_weights_1555865398.1238055_Apr22_5cls.h5'
class_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
model_type = "DenseNet121"

In [5]:
model_factory = ModelFactory()
model = model_factory.get_model( class_names,
                                model_name=model_type,
                                use_base_weights=True,
                                weights_path=model_file_path,
                                input_shape=(img_height, img_width, 3))
optimizer = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
load model weights_path ../weights/best_weights_1555982768.7076797.h5


In [16]:
from src.generator import CheXpertDataGenerator
valid_data = CheXpertDataGenerator(dataset_csv_file=valid_file,
            class_names=class_names,
            source_image_dir=data_dir,
            batch_size=1,
            target_size=(224, 224),
            augmenter=None,
            policy ='mixed',
        )

/Users/rishabhbhardwaj/Documents/personal/courses/cse6250/project/Chest-XRay-Classification/src/generator.py:73: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.x_path, y_df = df["Path"].as_matrix(), df[self.class_names]


In [17]:
pred = model.predict_generator(valid_data)

In [31]:
df = pd.read_csv(valid_file)
df = df[df['Frontal/Lateral']=='Frontal']
class_df = df[class_names]
gt = class_df.as_matrix()

/anaconda2/envs/cse6250/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [32]:
def computeAUROC (dataGT, dataPRED, classCount):
    outAUROC = []
    for i in range(classCount):
        try:
            outAUROC.append(roc_auc_score(dataGT[:, i], dataPRED[:, i]))
        except ValueError:
            pass
    return outAUROC

In [35]:
aurocIndividual = computeAUROC(gt, pred, 5)
aurocMean = np.array(aurocIndividual).mean()
print ('AUROC mean ', aurocMean)
for i in range (0, len(aurocIndividual)):
    print (class_names[i], ' ', aurocIndividual[i])

AUROC mean  0.48077689331389084
Atelectasis   0.5191601049868766
Cardiomegaly   0.500222816399287
Consolidation   0.46341911764705884
Edema   0.4635416666666667
Pleural Effusion   0.45754076086956524
